In [1]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel, TfidfModel

In [3]:
df = pd.read_csv('/content/drive/MyDrive/2023년 10월-11월 프로젝트/토큰화 진행 및 불용어 처리/2023년 기사(완료).csv')
df.drop(columns=['Unnamed: 0'],inplace=True)
df.reset_index(drop=True, inplace=True)
df

,date,url,category,tokenize_title,tokenize_content
0,2023-01-01T06:00:00,https://n.news.naver.com/mnews/article/421/000...,IT/과학,"['새해', '달라지는', '카카오', '먹통', '방지법', '시행', '콘텐츠'...","['데이터', '센터', '다중화', '의무', '콘텐츠', '세액', '공제', ..."
1,2023-01-01T12:50:00,https://n.news.naver.com/mnews/article/008/000...,경제,"['비', '대위', '체제', '끝낸', '카카오', '구상권', '청구', '본...","['카카오', '비상', '대책', '위원회', '체제', '종료', '하고', '..."
2,2023-01-01T15:12:00,https://n.news.naver.com/mnews/article/421/000...,경제,"['뉴스', '잇', '쥬', '카카오', '먹통', '사태', '보상', '등장'...","['카카오', '개인', '비즈니스', '파트너', '구분', '해', '일괄', ..."
3,2023-01-01T15:41:00,https://n.news.naver.com/mnews/article/015/000...,IT/과학,"['속', '속결', '마무리', '새해', '새롭게', '카카오', '비', '대...","['지난해', '초유', '먹통', '사태', '비상', '대책', '위원회', '..."
4,2023-01-01T15:54:00,https://n.news.naver.com/mnews/article/215/000...,IT/과학,"['비', '대위', '끝낸', '카카오', '상금', '청구', '전망']","['지난해', '초유', '먹통', '사태', '빚었던', '카카오', '비', '..."
...,...,...,...,...,...
5600,2023-10-24T21:40:00,https://n.news.naver.com/mnews/article/056/001...,경제,"['카카오', '법인', '처벌', '강수', '둔', '금감원', '발언', '배경']","['카카오', '창업', '김범수', '이사회', '의장', '금융감독원', '공개..."
5601,2023-10-24T22:01:00,https://n.news.naver.com/mnews/article/469/000...,IT/과학,"['카카오', '게임', '직원', '업데이트', '정보', '유출', '사실']","['카카오', '게임', '게임', '업데이트', '정보', '사전', '유출', ..."
5602,2023-10-24T22:29:00,https://n.news.naver.com/mnews/article/009/000...,경제,"['풍전등화', '카카오', '시세', '조작', '벌금형', '카뱅', '포기',...","['금감원', '칼춤', '예고', '카카오', '운명', '경제', '적', '이..."
5603,2023-10-24T23:01:00,https://n.news.naver.com/mnews/article/052/000...,경제,"['금감원', '카카오', '법인', '처벌', '검토', '송치']","['이복', '금융감독원장', '엔터테인먼트', '시세', '조종', '의혹', '..."


In [4]:
# 각 'tokenize_title'과 'tokenize_content'의 type이 str이므로 이를 list로 변환하기 위함
def str_list(token):
  str_list = token
  str_list = str_list.replace("[","").replace("]","").replace("'","").replace(" ","")
  result = str_list.split(",")
  return result

In [5]:
# title에 대한 LDA모델 적용
title_docs = []
for doc in df['tokenize_title']:
  title_docs.append(str_list(doc))
title_docs

[['새해', '달라지는', '카카오', '먹통', '방지법', '시행', '콘텐츠', '세액', '공제'],
 ['비', '대위', '체제', '끝낸', '카카오', '구상권', '청구', '본격', '화'],
 ['뉴스', '잇', '쥬', '카카오', '먹통', '사태', '보상', '등장', '현모', '대표', '최종', '후보', '낙점'],
 ['속', '속결', '마무리', '새해', '새롭게', '카카오', '비', '대위', '체제', '종료'],
 ['비', '대위', '끝낸', '카카오', '상금', '청구', '전망'],
 ['먹통', '사태', '카카오', '비', '대위', '종료', '상대', '소송', '전망'],
 ['먹통', '대란', '카카오', '비', '대위', '종료', '경영', '정상화', '돌입'],
 ['먹통', '피해', '보상', '마련', '카카오', '비', '대위', '체제', '종료'],
 ['이모티콘', '보상', '마련', '카카오', '구상권', '소송', '임박'],
 ['카카오', '대표', '미래', '이니셔티브', '센터', '고문'],
 ['공정위', '카카오', '엔터', '불공정', '혐의', '조사', '마치', '고', '상반기', '심의'],
 ['공정위', '카카오', '엔터', '불공정', '혐의', '조사', '마치', '고', '상반기', '심의'],
 ['용인시', '홈페이지', '네이버', '카카오', '로그인', '하세요'],
 ['먹통', '보상', '완료', '카카오', '경영', '집중', '상대로', '구상권'],
 ['게시판', '카카오', '가정', '쌀', '전달'],
 ['공정위', '카카오', '엔터', '불공정', '혐의', '조사', '마치', '고', '상반기', '심의'],
 ['이모티콘', '카카오', '보상', '책', '국민', '누구', '놀리나', '민심', '레이더'],
 ['네이버', '카카오', '새해', '웹툰', '돈', '벌

In [6]:
id2word_title = corpora.Dictionary(title_docs)
print(id2word_title)

Dictionary<6138 unique tokens: ['공제', '달라지는', '먹통', '방지법', '새해']...>


In [7]:
for value in id2word_title:
  print(value, id2word_title[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1138 본다
1139 약관
1140 집니다
1141 책임
1142 분실
1143 비단
1144 사건
1145 압수수색
1146 올려
1147 기념
1148 인사말
1149 상공
1150 포토
1151 덕담
1152 개방
1153 민간
1154 이용
1155 같은
1156 다른
1157 칼럼
1158 도로
1159 도운
1160 모빌리티
1161 이웃
1162 히어로즈
1163 딜
1164 일별
1165 테마
1166 프로모션
1167 구현
1168 러닝
1169 머신
1170 개발
1171 선봬
1172 옵스
1173 보상금
1174 서류
1175 접수
1176 제출
1177 절차
1178 예정
1179 착수
1180 현금
1181 모델
1182 아이
1183 학습
1184 게
1185 사장
1186 최선
1187 관리
1188 빠르게
1189 쉽고
1190 놀금
1191 오피스
1192 유지
1193 제도
1194 투
1195 고젝
1196 라오스
1197 상륙
1198 답정너
1199 전원
1200 회의
1201 한판
1202 바이오
1203 반도체
1204 뽑은
1205 펀드매니저
1206 근거리
1207 만나
1208 배송
1209 플러스
1210 영역
1211 개관
1212 도서관
1213 안산
1214 작은
1215 평화
1216 포레
1217 맥시
1218 치킨
1219 칸
1220 초읽기
1221 사용
1222 재생에너지
1223 전력
1224 충당
1225 기준
1226 모임
1227 붙나
1228 토스
1229 통장
1230 결과
1231 안도
1232 건설
1233 한화
1234 클립
1235 금리
1236 꺾이나
1237 한풀
1238 환호
1239 인상
1240 호점
1241 조성
1242 기본
1243 대형
1244 손잡고
1245 싸네
1246 아이엠
1247 중형
1248 타타
1249 문
1250 열어
1251 끝
1252 보인다
1253 웃음꽃
1254 무명
12

In [8]:
corpus_TDM = []
for doc in title_docs:
  result = id2word_title.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)],
 [(7, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1)],
 [(2, 1),
  (7, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1)],
 [(4, 1),
  (7, 1),
  (11, 1),
  (13, 1),
  (15, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1)],
 [(7, 1), (10, 1), (11, 1), (13, 1), (14, 1), (33, 1), (34, 1)],
 [(2, 1),
  (7, 1),
  (11, 1),
  (13, 1),
  (22, 1),
  (32, 1),
  (34, 1),
  (35, 1),
  (36, 1)],
 [(2, 1),
  (7, 1),
  (11, 1),
  (13, 1),
  (32, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1)],
 [(2, 1),
  (7, 1),
  (11, 1),
  (13, 1),
  (15, 1),
  (21, 1),
  (32, 1),
  (41, 1),
  (42, 1)],
 [(7, 1), (9, 1), (21, 1), (36, 1), (41, 1), (43, 1), (44, 1)],
 [(7, 1), (19, 1), (45, 1), (46, 1), (47, 1), (48, 1)],
 [(7, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),

In [9]:
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_title, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.030*"포토" + 0.025*"의장" + 0.019*"법인" + 0.014*"송치" + 0.011*"이복" + 0.007*"쇼핑" + 0.007*"할인" + 0.007*"현실" + 0.006*"김범수" + 0.006*"네이버"
1 : 0.012*"회사" + 0.012*"심사" + 0.008*"공정위" + 0.007*"약세" + 0.007*"김범수" + 0.006*"사전" + 0.006*"부당" + 0.005*"선정" + 0.005*"게임" + 0.005*"소비자"
2 : 0.065*"시세" + 0.065*"조종" + 0.057*"의혹" + 0.037*"금감원" + 0.022*"투자" + 0.021*"김범수" + 0.018*"대표" + 0.013*"리스크" + 0.010*"수사" + 0.010*"직원"
3 : 0.051*"압수수색" + 0.037*"혐의" + 0.022*"금감원" + 0.021*"시세" + 0.021*"조종" + 0.018*"검찰" + 0.010*"수수료" + 0.009*"청구" + 0.009*"엔터" + 0.007*"붕괴"
4 : 0.006*"출신" + 0.006*"공인" + 0.005*"페이" + 0.005*"소상" + 0.005*"서비스" + 0.004*"지원" + 0.004*"베스트" + 0.004*"뚝" + 0.004*"라인" + 0.004*"주가"
5 : 0.046*"창업" + 0.045*"김범수" + 0.024*"센터" + 0.019*"데이터" + 0.017*"금감원" + 0.013*"신고" + 0.011*"위기" + 0.008*"다음" + 0.006*"청구" + 0.006*"사무실"
6 : 0.041*"총괄" + 0.018*"조사" + 0.018*"대표" + 0.016*"김범수" + 0.009*"적극" + 0.008*"금감원" + 0.008*"심사" + 0.008*"창업" + 0.008*"환영" + 0.007*"상반기"
7 : 0.020*"통보" + 0.014*"먹통" + 0.009*"사태" + 0.009*"프렌즈" +

In [10]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(title 부분)
lda[corpus_TFIDF[0]]

[(0, 0.0270849),
 (1, 0.027151752),
 (2, 0.02733867),
 (3, 0.02773746),
 (4, 0.027301684),
 (5, 0.7545647),
 (6, 0.027445903),
 (7, 0.02708803),
 (8, 0.027197931),
 (9, 0.027088992)]

In [11]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.03373173), (1, 0.033738434), (2, 0.03373694), (3, 0.03373095), (4, 0.033738274), (5, 0.03374099), (6, 0.03372954), (7, 0.033729255), (8, 0.30478328), (9, 0.42534065)]
[(0, 0.028827189), (1, 0.028826753), (2, 0.50346905), (3, 0.02883218), (4, 0.028826699), (5, 0.028825983), (6, 0.2659064), (7, 0.028826876), (8, 0.028826734), (9, 0.02883213)]
[(0, 0.029592382), (1, 0.029596658), (2, 0.02961364), (3, 0.029590344), (4, 0.7336337), (5, 0.029594291), (6, 0.02959625), (7, 0.029594095), (8, 0.029595442), (9, 0.029593209)]
[(0, 0.027200168), (1, 0.027200112), (2, 0.027200092), (3, 0.027200833), (4, 0.027198879), (5, 0.027199233), (6, 0.027200278), (7, 0.027199425), (8, 0.027199665), (9, 0.75520134)]
[(0, 0.03122098), (1, 0.031229516), (2, 0.03123206), (3, 0.031220764), (4, 0.031228278), (5, 0.031231163), (6, 0.03121914), (7, 0.03122153), (8, 0.22215705), (9, 0.5280395)]
[(0, 0.027789822), (1, 0.027805634), (2, 0.02782118), (3, 0.17090887), (4, 0.027796

In [12]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(title)'] = dominant_topic[0]
  df.loc[i, 'topic_details(title)'] = dominant_topic[1]
df

,date,url,category,tokenize_title,tokenize_content,topic(title),topic_details(title)
0,2023-01-01T06:00:00,https://n.news.naver.com/mnews/article/421/000...,IT/과학,"['새해', '달라지는', '카카오', '먹통', '방지법', '시행', '콘텐츠'...","['데이터', '센터', '다중화', '의무', '콘텐츠', '세액', '공제', ...",5.0,0.700204
1,2023-01-01T12:50:00,https://n.news.naver.com/mnews/article/008/000...,경제,"['비', '대위', '체제', '끝낸', '카카오', '구상권', '청구', '본...","['카카오', '비상', '대책', '위원회', '체제', '종료', '하고', '...",5.0,0.760781
2,2023-01-01T15:12:00,https://n.news.naver.com/mnews/article/421/000...,경제,"['뉴스', '잇', '쥬', '카카오', '먹통', '사태', '보상', '등장'...","['카카오', '개인', '비즈니스', '파트너', '구분', '해', '일괄', ...",7.0,0.789344
3,2023-01-01T15:41:00,https://n.news.naver.com/mnews/article/015/000...,IT/과학,"['속', '속결', '마무리', '새해', '새롭게', '카카오', '비', '대...","['지난해', '초유', '먹통', '사태', '비상', '대책', '위원회', '...",5.0,0.696013
4,2023-01-01T15:54:00,https://n.news.naver.com/mnews/article/215/000...,IT/과학,"['비', '대위', '끝낸', '카카오', '상금', '청구', '전망']","['지난해', '초유', '먹통', '사태', '빚었던', '카카오', '비', '...",3.0,0.733463
...,...,...,...,...,...,...,...
5600,2023-10-24T21:40:00,https://n.news.naver.com/mnews/article/056/001...,경제,"['카카오', '법인', '처벌', '강수', '둔', '금감원', '발언', '배경']","['카카오', '창업', '김범수', '이사회', '의장', '금융감독원', '공개...",6.0,0.517432
5601,2023-10-24T22:01:00,https://n.news.naver.com/mnews/article/469/000...,IT/과학,"['카카오', '게임', '직원', '업데이트', '정보', '유출', '사실']","['카카오', '게임', '게임', '업데이트', '정보', '사전', '유출', ...",3.0,0.581949
5602,2023-10-24T22:29:00,https://n.news.naver.com/mnews/article/009/000...,경제,"['풍전등화', '카카오', '시세', '조작', '벌금형', '카뱅', '포기',...","['금감원', '칼춤', '예고', '카카오', '운명', '경제', '적', '이...",1.0,0.496321
5603,2023-10-24T23:01:00,https://n.news.naver.com/mnews/article/052/000...,경제,"['금감원', '카카오', '법인', '처벌', '검토', '송치']","['이복', '금융감독원장', '엔터테인먼트', '시세', '조종', '의혹', '...",0.0,0.565989


In [13]:
# 토픽별 비중 확인
df['topic(title)'].unique()

array([5., 7., 3., 0., 6., 8., 4., 2., 1., 9.])

In [14]:
df['topic(title)'].value_counts()

2.0    887
6.0    687
5.0    622
7.0    577
8.0    551
3.0    525
1.0    515
9.0    491
0.0    380
4.0    370
Name: topic(title), dtype: int64

In [15]:
# content에 대한 LDA모델 적용
content_docs = []
for doc in df['tokenize_content']:
  content_docs.append(str_list(doc))
content_docs

Output hidden; open in https://colab.research.google.com to view.

In [16]:
id2word_content = corpora.Dictionary(content_docs)
print(id2word_content)

Dictionary<32782 unique tokens: ['가능해진다', '간섭', '간소화', '갖추면', '같은']...>


In [17]:
for value in id2word_content:
  print(value, id2word_content[value])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
27782 뚝뚝
27783 말까
27784 말씀드리자면
27785 멀죠
27786 미비했다고
27787 살펴보면요
27788 수사의뢰
27789 아니나
27790 업도
27791 올랐는데요
27792 올리려면
27793 잠잠한가
27794 재위
27795 짤막하게
27796 탁하는
27797 했더니
27798 스며든다
27799 대출받았다
27800 박준석
27801 논개
27802 부울경
27803 산태
27804 진주
27805 김광신
27806 구글지도
27807 꼽았으
27808 넓지
27809 뒤진
27810 분야였는데
27811 우월했다
27812 종결자
27813 치율
27814 냄으로써
27815 건강검진
27816 단양
27817 단양군
27818 디아
27819 받기라도
27820 사카모토
27821 신월동
27822 영화음악
27823 오은지
27824 자재
27825 정진욱
27826 제천
27827 제천시
27828 찬송
27829 최영준
27830 충주시
27831 충청북도
27832 타계
27833 티젠
27834 해드렸는데요
27835 해드렸습니다
27836 글래스
27837 네스
27838 다비
27839 로더
27840 로레알
27841 마르
27842 메종
27843 바뇨
27844 반품
27845 부르켓
27846 시세이도
27847 시슬리
27848 영자
27849 용이하면서도
27850 이병희
27851 정품
27852 직매
27853 꾸미기로
27854 끝말잇기
27855 나빠졌을
27856 늦었지만
27857 돌아보면
27858 뚜렷하게
27859 뚜렷해
27860 무겁지만
27861 시달렸던
27862 아니어도
27863 유력하지만
27864 즐기기
27865 착한다면
27866 팅할
27867 퍼뜨릴
27868 품겠다는
27869 활발해지면
27870 개그우먼
27871 구워내어
27872 놀라는
27873 놀라로
27874 놀라를
27875 누리시길

In [18]:
corpus_TDM = []
for doc in content_docs:
  result = id2word_content.doc2bow(doc)
  corpus_TDM.append(result)

corpus_TDM # index, 등장 횟수 (순서는 index 순으로 sorting해서 출력됨)

Output hidden; open in https://colab.research.google.com to view.

In [19]:
# 토큰화 진행 및 불용어 처리에서 불용어에 대해 업데이트 해줘야함!!!
tfidf = TfidfModel(corpus_TDM)
corpus_TFIDF = tfidf[corpus_TDM]

n = 10 # 토픽갯수 - 변경가능
lda = LdaModel(corpus=corpus_TFIDF, id2word=id2word_content, num_topics=n, random_state=100)

for t in lda.print_topics():
  print(t[0],":",t[1])

0 : 0.003*"장애" + 0.002*"재난" + 0.002*"데이터" + 0.002*"센터" + 0.002*"서비스" + 0.002*"안산" + 0.002*"서버" + 0.001*"화재" + 0.001*"재해" + 0.001*"먹통"
1 : 0.004*"소송" + 0.004*"받기" + 0.003*"여의도" + 0.003*"출석" + 0.003*"손해배상" + 0.002*"서민" + 0.002*"이니셔티브" + 0.002*"청구" + 0.002*"남부" + 0.002*"센터"
2 : 0.009*"압수수색" + 0.003*"집회" + 0.002*"경찰" + 0.002*"행진" + 0.002*"수사" + 0.002*"에스엠" + 0.002*"성남시" + 0.002*"배틀" + 0.002*"판교" + 0.002*"관여"
3 : 0.013*"조종" + 0.012*"시세" + 0.010*"혐의" + 0.009*"금감원" + 0.009*"의혹" + 0.008*"특사" + 0.007*"하이브" + 0.007*"자본시장" + 0.007*"공개매수" + 0.007*"금융감독원"
4 : 0.003*"남부" + 0.003*"증거" + 0.002*"출석" + 0.002*"경찰" + 0.002*"스코어" + 0.002*"했습니다" + 0.002*"의장" + 0.002*"하이브" + 0.002*"뉴스" + 0.001*"작가"
5 : 0.004*"뱅크" + 0.003*"대표" + 0.002*"서비스" + 0.002*"영장" + 0.002*"수사" + 0.002*"투자" + 0.002*"결제" + 0.002*"사업" + 0.002*"모빌리티" + 0.002*"택시"
6 : 0.003*"매출" + 0.003*"데이터" + 0.003*"영등포구" + 0.003*"센터" + 0.003*"안산" + 0.003*"영업" + 0.002*"이익" + 0.002*"게임" + 0.002*"광고" + 0.002*"안산시"
7 : 0.004*"집회" + 0.002*"노조" + 0.002*"린지" + 0

In [20]:
# 반복문 돌려서 기사에 토픽 일일이 배정해줘야 함(content 부분)
lda[corpus_TFIDF[0]]

[(0, 0.49129635), (5, 0.4331589)]

In [21]:
for i in range(len(corpus_TFIDF)):
  print(lda[corpus_TFIDF[i]])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[(0, 0.010695427), (1, 0.01072604), (2, 0.0106895575), (3, 0.010699067), (4, 0.010695829), (5, 0.010731393), (6, 0.9036907), (7, 0.01069371), (8, 0.010690465), (9, 0.010687849)]
[(0, 0.011741385), (1, 0.011741167), (2, 0.011740606), (3, 0.011741731), (4, 0.011741567), (5, 0.8943255), (6, 0.011742125), (7, 0.011741576), (8, 0.011741016), (9, 0.011743318)]
[(3, 0.42591), (4, 0.23433259), (5, 0.2725554)]
[(0, 0.010763947), (1, 0.0107721), (2, 0.010762529), (3, 0.010769362), (4, 0.010766976), (5, 0.65643454), (6, 0.010764831), (7, 0.010764787), (8, 0.010763093), (9, 0.25743783)]
[(6, 0.8385885), (7, 0.09713073)]
[(0, 0.010586178), (1, 0.010584878), (2, 0.01058434), (3, 0.010595983), (4, 0.010611299), (5, 0.010590527), (6, 0.9046934), (7, 0.010583942), (8, 0.010584893), (9, 0.010584517)]
[(5, 0.21281496), (6, 0.7086204)]
[(0, 0.010484002), (1, 0.010485502), (2, 0.010484084), (3, 0.01048589), (4, 0.010485951), (5, 0.010490685), (6, 0.76245284), (7, 0.01048

In [22]:
for i in range(len(corpus_TFIDF)):
  dominant_topic = sorted(lda[corpus_TFIDF[i]], key=lambda x: x[1], reverse=True)[0]
  df.loc[i, 'topic(content)'] = dominant_topic[0]
  df.loc[i, 'topic_details(content)'] = dominant_topic[1]
df

,date,url,category,tokenize_title,tokenize_content,topic(title),topic_details(title),topic(content),topic_details(content)
0,2023-01-01T06:00:00,https://n.news.naver.com/mnews/article/421/000...,IT/과학,"['새해', '달라지는', '카카오', '먹통', '방지법', '시행', '콘텐츠'...","['데이터', '센터', '다중화', '의무', '콘텐츠', '세액', '공제', ...",5.0,0.700204,0.0,0.491284
1,2023-01-01T12:50:00,https://n.news.naver.com/mnews/article/008/000...,경제,"['비', '대위', '체제', '끝낸', '카카오', '구상권', '청구', '본...","['카카오', '비상', '대책', '위원회', '체제', '종료', '하고', '...",5.0,0.760781,5.0,0.478296
2,2023-01-01T15:12:00,https://n.news.naver.com/mnews/article/421/000...,경제,"['뉴스', '잇', '쥬', '카카오', '먹통', '사태', '보상', '등장'...","['카카오', '개인', '비즈니스', '파트너', '구분', '해', '일괄', ...",7.0,0.789344,5.0,0.675523
3,2023-01-01T15:41:00,https://n.news.naver.com/mnews/article/015/000...,IT/과학,"['속', '속결', '마무리', '새해', '새롭게', '카카오', '비', '대...","['지난해', '초유', '먹통', '사태', '비상', '대책', '위원회', '...",5.0,0.696013,5.0,0.521845
4,2023-01-01T15:54:00,https://n.news.naver.com/mnews/article/215/000...,IT/과학,"['비', '대위', '끝낸', '카카오', '상금', '청구', '전망']","['지난해', '초유', '먹통', '사태', '빚었던', '카카오', '비', '...",3.0,0.733463,5.0,0.497079
...,...,...,...,...,...,...,...,...,...
5600,2023-10-24T21:40:00,https://n.news.naver.com/mnews/article/056/001...,경제,"['카카오', '법인', '처벌', '강수', '둔', '금감원', '발언', '배경']","['카카오', '창업', '김범수', '이사회', '의장', '금융감독원', '공개...",6.0,0.517432,3.0,0.533242
5601,2023-10-24T22:01:00,https://n.news.naver.com/mnews/article/469/000...,IT/과학,"['카카오', '게임', '직원', '업데이트', '정보', '유출', '사실']","['카카오', '게임', '게임', '업데이트', '정보', '사전', '유출', ...",3.0,0.581949,5.0,0.707408
5602,2023-10-24T22:29:00,https://n.news.naver.com/mnews/article/009/000...,경제,"['풍전등화', '카카오', '시세', '조작', '벌금형', '카뱅', '포기',...","['금감원', '칼춤', '예고', '카카오', '운명', '경제', '적', '이...",1.0,0.496321,3.0,0.498406
5603,2023-10-24T23:01:00,https://n.news.naver.com/mnews/article/052/000...,경제,"['금감원', '카카오', '법인', '처벌', '검토', '송치']","['이복', '금융감독원장', '엔터테인먼트', '시세', '조종', '의혹', '...",0.0,0.565989,3.0,0.763931


In [23]:
# 토픽별 비중 확인
df['topic(content)'].unique()

array([0., 5., 1., 2., 4., 6., 8., 3., 7., 9.])

In [24]:
df['topic(content)'].value_counts()

5.0    2881
3.0    1207
6.0     821
1.0     178
4.0     175
8.0     134
0.0      92
9.0      77
2.0      27
7.0      13
Name: topic(content), dtype: int64

In [25]:
# CSV 파일로 저장
csv_filename = "/content/drive/MyDrive/2023년 10월-11월 프로젝트/LDA모델 적용/news_2023.csv"
df.to_csv(csv_filename, index=False)